### OpenCV-Python tests
https://github.com/jagracar/OpenCV-python-tests/tree/master/OpenCV-tutorials/cameraCalibration

#### Camera Calibration

In [49]:
import numpy as np
import cv2
import glob

# Define the chess board rows and columns
rows = 7
cols = 6

# Image directory
img_dir = 'C:/Users/markl/startup/raw-images/Calibration/'

# Set the termination criteria for the corner sub-pixel algorithm
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.001)

# Prepare the object points: (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0). They are the same for all images
objectPoints = np.zeros((rows * cols, 3), np.float32)
objectPoints[:, :2] = np.mgrid[0:rows, 0:cols].T.reshape(-1, 2)

# Create the arrays to store the object points and the image points
objectPointsArray = []
imgPointsArray = []

# Loop over the image files
for path in glob.glob(img_dir+'L_Image*.jpg'):
    # Load the image and convert it to gray scale
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (rows, cols), None)

    # Make sure the chess board pattern was found in the image
    if ret:
        # Refine the corner position
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        
        # Add the object points and the image points to the arrays
        objectPointsArray.append(objectPoints)
        imgPointsArray.append(corners)

        # Draw the corners on the image
        cv2.drawChessboardCorners(img, (rows, cols), corners, ret)
    
    # Display the image
    cv2.imshow('chess board', img)
    cv2.waitKey(500)

# Calibrate the camera and save the results
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPointsArray, imgPointsArray, gray.shape[::-1], None, None)
np.savez(img_dir+'calib.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

# Print the camera calibration error
error = 0

for i in range(len(objectPointsArray)):
    imgPoints, _ = cv2.projectPoints(objectPointsArray[i], rvecs[i], tvecs[i], mtx, dist)
    error += cv2.norm(imgPointsArray[i], imgPoints, cv2.NORM_L2) / len(imgPoints)

print("Total error: ", error / len(objectPointsArray))

# Load one of the test images
img = cv2.imread(img_dir+'L_Image20.jpg')
h, w = img.shape[:2]

# Obtain the new camera matrix and undistort the image
newCameraMtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
undistortedImg = cv2.undistort(img, mtx, dist, None, newCameraMtx)

# Crop the undistorted image
x, y, w, h = roi
undistortedImg = undistortedImg[y:y + h, x:x + w]

# Display the final result
cv2.imshow('chess board', np.hstack((img, undistortedImg)))
cv2.waitKey(0)
cv2.destroyAllWindows()

Total error:  0.04631991702911602


ValueError: all the input array dimensions except for the concatenation axis must match exactly

#### Pose Estimation

In [3]:
'''
 Based on the following tutorial:
   http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_pose/py_pose.html
'''

import numpy as np
import cv2
import glob

# Image directory
img_dir = 'C:/Users/markl/startup/raw-images/Calibration/'

# This function draws lines joining the given image points to the first chess board corner
def draw(img, corners, imgPoints):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgPoints[0].ravel()), (255, 0, 0), 5)
    img = cv2.line(img, corner, tuple(imgPoints[1].ravel()), (0, 255, 0), 5)
    img = cv2.line(img, corner, tuple(imgPoints[2].ravel()), (0, 0, 255), 5)
    return img

# Load the camera calibration data
with np.load(img_dir+'calib.npz') as calibData:
    mtx, dist, rvecs, tvecs = [calibData[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]

# Define the chess board rows and columns
rows = 7
cols = 6


# Set the termination criteria for the corner sub-pixel algorithm
criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.001)

# Prepare the object points: (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0). They are the same for all images
objectPoints = np.zeros((rows * cols, 1, 3), np.float32)
objectPoints[:, :, :2] = np.mgrid[0:rows, 0:cols].T.reshape(-1, 1, 2)

# Create the axis points
axisPoints = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1, 3)

# Loop over the image files
for path in glob.glob(img_dir+'L_Image*.jpg'):
    # Load the image and convert it to gray scale
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (rows, cols), None)

    # Make sure the chess board pattern was found in the image
    if ret:
        # Refine the corner position
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        # Find the rotation and translation vectors
        val, rvecs, tvecs, inliers = cv2.solvePnPRansac(objectPoints, corners, mtx, dist)

        # Project the 3D axis points to the image plane
        axisImgPoints, jac = cv2.projectPoints(axisPoints, rvecs, tvecs, mtx, dist)
        
        # Draw the axis lines
        img = draw(img, corners, axisImgPoints)
    
    # Display the image
    cv2.imshow('chess board', img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

 #### Epipolar Geometry

In [3]:
'''
 Based on the following tutorial:
   http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_epipolar_geometry/py_epipolar_geometry.html
'''

import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib


img_dir = 'C:/Users/markl/startup/raw-images/Calibration/'

# Load the left and right images in gray scale
imgLeft = cv2.imread(img_dir+'L_Image20.jpg',0)
imgRight = cv2.imread(img_dir+'R_Image20.jpg',0)

# Detect the SIFT key points and compute the descriptors for the two images
sift = cv2.xfeatures2d.SIFT_create()
keyPointsLeft, descriptorsLeft = sift.detectAndCompute(imgLeft, None)
keyPointsRight, descriptorsRight = sift.detectAndCompute(imgRight, None)

# Create FLANN matcher object
FLANN_INDEX_KDTREE = 0
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
searchParams = dict(checks=50)
flann = cv2.FlannBasedMatcher(indexParams, searchParams)

# Match the descriptors (this crashes in OpenCV3.1)
# See https://github.com/Itseez/opencv/issues/5667
matches = flann.knnMatch(descriptorsLeft, descriptorsRight, k=2)

# Apply ratio test
goodMatches = []
ptsLeft = []
ptsRight = []

for m, n in matches:
    if m.distance < 0.8 * n.distance:
        goodMatches.append([m])
        ptsLeft.append(keyPointsLeft[m.trainIdx].pt)
        ptsRight.append(keyPointsRight[n.trainIdx].pt)

ptsLeft = np.int32(ptsLeft)
ptsRight = np.int32(ptsRight)
F, mask = cv2.findFundamentalMat(ptsLeft, ptsRight, cv2.FM_LMEDS)

# We select only inlier points
ptsLeft = ptsLeft[mask.ravel() == 1]
ptsRight = ptsRight[mask.ravel() == 1]

def drawlines(img1, img2, lines, pts1, pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r, c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2] / r[1] ])
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1] ])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 1)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
    return img1, img2

# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
linesLeft = cv2.computeCorrespondEpilines(ptsRight.reshape(-1, 1, 2), 2, F)
linesLeft = linesLeft.reshape(-1, 3)
img5, img6 = drawlines(imgLeft, imgRight, linesLeft, ptsLeft, ptsRight)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
linesRight = cv2.computeCorrespondEpilines(ptsLeft.reshape(-1, 1, 2), 1, F)
linesRight = linesRight.reshape(-1, 3)
img3, img4 = drawlines(imgRight, imgLeft, linesRight, ptsRight, ptsLeft)

plt.subplot(121), plt.imshow(img5)
plt.subplot(122), plt.imshow(img3)
plt.show()

Using matplotlib backend: Qt5Agg


#### Depth Map

In [69]:
'''
 Based on the following tutorial:
   http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_depthmap/py_depthmap.html
'''

import numpy as np
import cv2

# Load img directory 2
img_dir2 = 'C:/Users/markl/startup/raw-images/Disparity/'
# Load the left and right images in gray scale
imgLeft = cv2.imread(img_dir2+'L_Image18.jpg',0)
imgRight = cv2.imread(img_dir2+'R_Image18.jpg',0)


#for disnum in range(0,17):
#    for blksz in range(7,21,2):
        
        #print('numDisparities: {}, blockSize: {}'.format(disnum,blksz))
        
        # Initialize the stereo block matching object 
        stereo = cv2.StereoBM_create(numDisparities=128, blockSize=21)
        stereo.setMinDisparity(4)
        stereo.setNumDisparities(128)
        stereo.setBlockSize(21)
        stereo.setSpeckleRange(16)
        stereo.setSpeckleWindowSize(45)
        # Compute the disparity image
        disparity = stereo.compute(imgLeft, imgRight)

        # Normalize the image for representation
        min = disparity.min()
        max = disparity.max()
        disparity = np.uint8(255 * (disparity - min) / (max - min))

        # Display the result
        #cv2.imshow('Disparity -  numDisparities: {}, blockSize: {}'.format(disnum,blksz), np.hstack((imgLeft, imgRight, disparity)))
        cv2.imshow('Disparity -  numDisparities: {}, blockSize: {}'.format(disnum,blksz), disparity)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()

In [54]:
def disparity(self):
        matcher = cv2.StereoBM_create(1024, 7)
        disparity = matcher.compute(cv2.cvtColor(self.images[0], cv2.COLOR_BGR2GRAY),
                                    cv2.cvtColor(self.images[1], cv2.COLOR_BGR2GRAY))
        self.process_output(disparity) 

def readyStereoBM(roi1, roi2):
    stereobm = cv2.StereoBM_create(numDisparities=112, blockSize=31)
    stereobm.setPreFilterSize(31)#41
    stereobm.setPreFilterType(cv2.STEREO_BM_PREFILTER_NORMALIZED_RESPONSE)
    stereobm.setPreFilterCap(31)
    stereobm.setTextureThreshold(10)
    stereobm.setMinDisparity(0)
    stereobm.setSpeckleWindowSize(100)
    stereobm.setSpeckleRange(64)
    stereobm.setUniquenessRatio(0)
    stereobm.setROI1(roi1)
    stereobm.setROI1(roi2)
    return stereobm 